## Import libraries

In [2]:
import urllib.request
import re
import importlib
import tiktoken

## Get text

In [3]:
url = "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/refs/heads/main/ch02/01_main-chapter-code/the-verdict.txt"
file_path = "the-verdict.txt"
urllib.request.urlretrieve(url=url, filename=file_path)

('the-verdict.txt', <http.client.HTTPMessage at 0x7be192357760>)

In [4]:
with open("the-verdict.txt", mode="r", encoding="utf-8") as f:
    raw_text = f.read()
print(f"Total number of characters is {len(raw_text)}")
print(raw_text[:30])

Total number of characters is 20479
I HAD always thought Jack Gisb


## Split words into tokens

In [5]:
# Using a regex, split words. First part is taking special characters as separate
# tokens and second \s is takins words as tokens
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
# Removing whitespaces
preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


## Change tokens into integers

In [6]:
# Sort word ans set takes only unique values
all_words = sorted(set(preprocessed))
# Get vocab size
vocab_size = len(all_words)

print(vocab_size)

1130


In [7]:
vocab = {token: integer for integer, token in enumerate(all_words)}

In [8]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        # Vocab is dictionary (ID:Word, ID:Word....)
        self.str_to_int = vocab
        # Switch to {Word:ID...} structure
        self.int_to_str = {i: s for s, i in vocab.items()}

    def encode(self, text):
        # Same as before. Special characters plus words
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        # Remove whitespace chars
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        # Convert to integers
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r"\1", text)
        return text

In [9]:
simple_tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know," 
           Mrs. Gisburn said with pardonable pride."""
ids = simple_tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [10]:
simple_tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [11]:
simple_tokenizer.decode(simple_tokenizer.encode(text))

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [12]:
simple_tokenizer.encode("Hello")

KeyError: 'Hello'

This is the problem of word tokenizes. You need to have all words in corpus at the beginning, or you have to substitute them as bellow. We add <|unk|> for unknown tokens and <|endoftext|> as end of sentece etc. 

In [13]:
all_tokens = sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token: integer for integer, token in enumerate(all_tokens)}

In our vocab we added at the end these two new tokens. 

In [14]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


And lets reimplement Tokenizes by adding there situation of unknown words

In [15]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i: s for s, i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [item if item in self.str_to_int else "<|unk|>" for item in preprocessed]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.:;?!"()\'])', r"\1", text)
        return text

In [16]:
simple_tokenizer = SimpleTokenizerV2(vocab)

Lets try this again

In [17]:
simple_tokenizer.encode("Hello")

[1131]

In [18]:
simple_tokenizer.decode(simple_tokenizer.encode("Hello"))

'<|unk|>'

Problem is that later we can have many unknown word or their permutations etc. It would cause significant decrease of performace. Thats why there is use of BytePair Necoding techniques.

## BytePair encoding

BytePair technique when trained start with replacing single byte with integer. Then it start replacing most common groups of integer with another integer, so then you have a combination of bytes, part of words or even whole words. 

In [19]:
byte_pair_tokenizer = tiktoken.get_encoding("gpt2")

In [20]:
text = "Hello, do you like tea? <|endoftext|> In the sunlit terraces" "of someunknownPlace."

integers = byte_pair_tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)
print(len(integers))

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]
20


You can see, that we have 20 integers from our original text. Lets compare it with our previous tokenizes. 

In [22]:
integers_simple = simple_tokenizer.encode(text)
print(integers_simple)
print(len(integers_simple))

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 1131, 1131, 7]
14


20 tokens in BytePair coding compare to 14 when you need to have defined all words. Pretty good huh? Try a special character and decoding.

In [25]:
byte_pair_tokenizer.encode("<|endoftext|>", allowed_special={"<|endoftext|>"})

[50256]

In [26]:
strings = byte_pair_tokenizer.decode(integers)

print(strings)

Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.
